In [ ]:
import pandas as pd

In [ ]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

app = FastAPI()

# Cargar el archivo CSV y crear la matriz de géneros
archivo = r'Datasets/csv/steam_games_normalizado.csv'
df_games = pd.read_csv(archivo)
df_subset = df_games.head(20000)  # Hacemos un recorte más pequeño del dataframe

# Usamos CountVectorizer para convertir los géneros a una matriz de conteo
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(', '))
genres_matrix = vectorizer.fit_transform(df_subset['genres'])

# Calcular la matriz de similitud del coseno
cosine_sim = cosine_similarity(genres_matrix)

@app.get("/recomendaciones/")
def obtener_similitud_juego(nombre_juego: str, n_recomendaciones: int = 5):
    """
    Retorna una lista de juegos similares al nombre de juego dado, utilizando la similitud de coseno en los géneros.

    Parámetros:
    - nombre_juego: Nombre del juego para el cual buscar recomendaciones (str).
    - n_recomendaciones: Número de recomendaciones a devolver (int, por defecto 5).

    Retorna:
    - JSONResponse con una lista de juegos recomendados.
    """
    try:
        # Verificar si el juego existe en el dataframe
        if nombre_juego not in df_subset['app_name'].values:
            return JSONResponse(
                content={"error": f"El juego '{nombre_juego}' no se encuentra en la base de datos."},
                status_code=404
            )

        # Obtener el índice del juego especificado
        idx = df_subset[df_subset['app_name'] == nombre_juego].index[0]

        # Obtener el vector del juego especificado y calcular las similitudes
        juego_vector = genres_matrix[idx]
        similitudes = cosine_similarity(juego_vector, genres_matrix).flatten()

        # Ordenar los índices de juegos similares y obtener los n_recomendaciones más altos
        indices_similares = similitudes.argsort()[-(n_recomendaciones + 1):-1][::-1]

        # Obtener los nombres de los juegos similares
        juegos_similares = df_subset.iloc[indices_similares]['app_name'].tolist()

        # Preparar la respuesta como JSON
        result = {
            "Juego Base": nombre_juego,
            "Recomendaciones": juegos_similares
        }

        return JSONResponse(content=result)

    except Exception as e:
        return JSONResponse(
            content={"error": f"Ocurrió un error: {str(e)}"},
            status_code=500
        )
